In [2]:
import os
import argparse
import time
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
import warnings
warnings.filterwarnings("ignore")
from utils import Data_utility, makeOptimizer
import torch
from tqdm import tqdm
from models import lob, LSTNet
from trainer import origin_train, origin_eval, evaluation, calibration_train, calibration_val
from utils import EarlyStopping, save_model, load_model
from pathlib import Path
import torch.nn as nn
from torchinfo import summary
import wandb

print("hello world")

import matplotlib.pyplot as plt

hello world


In [24]:

parser = argparse.ArgumentParser(description='PyTorch Time series forecasting')
parser.add_argument('--id', type=str, default="159920", help='location of the data file')
parser.add_argument('--model', type=str, default='LSTNet',  help='')
parser.add_argument('--window', type=int, default=50, help='window size')
parser.add_argument('--horizon', type=int, default=1)
parser.add_argument('--normalize', type=int, default=2)
parser.add_argument('--highway_window', type=int, default=3, help='The window size')
parser.add_argument('--kwindow', type=int, default=5, help='The window size')

parser.add_argument('-n_rnns', type=int, default=1, help='depth of the RNN model')
parser.add_argument('-rnn_hid_size', type=int, default=64)
parser.add_argument('--rnn_layers', type=int, default=1, help='number of RNN hidden layers')
parser.add_argument('-fc_hid_size', type=int, default=24)
parser.add_argument('--dropout', type=float, default=0.1, help='dropout applied to layers (0 = no dropout)')
parser.add_argument('--output_fun', type=str, default='tanh')
parser.add_argument('--kernel_type', type=str, default='lob', help='Model type')
parser.add_argument('--variance', type=bool, default=True, help='Variance')
parser.add_argument('--prediction_window_size', type=int, default=1, help='prediction')
parser.add_argument('--confidence', type=int, default=95, help='clip gradient')
parser.add_argument('--hidRNN', type=int, default=300, help='number of RNN hidden units each layer')
parser.add_argument('--hidCNN', type=int, default=300, help='number of CNN hidden units (channels)')
parser.add_argument('--CNN_kernel', type=int, default=3, help='the kernel size of the CNN layers')

parser.add_argument('-lr', type=float, default=1e-4)
parser.add_argument('-epochs', type=int, default=1000)
parser.add_argument('-epochs_mmd', type=int, default=10)
parser.add_argument('-batch_size', type=int, default=256)
parser.add_argument('-lh_scale', type=float, default=1.,
                    help="likelihood scale, shouldn't need tuning")
parser.add_argument('--seed', type=int, default=1111,help='random seed')
parser.add_argument('-gpu', type=str, default="6")
parser.add_argument('--model_path', type=str, default='./save',
                    help='path to save checkpoints (default: None)')
parser.add_argument('--patience', type=int, default=50, help='model path')
parser.add_argument('--delta', type=int, default=0, help='patience')
parser.add_argument('--verbose', type=bool, default=False, help='print patience information')
parser.add_argument('--grad_clip', type=int, default=10, help='clip gradient')
parser.add_argument('--calib', type=bool, default=True, help='calibration')
parser.add_argument('--weight-decay', type=float, default=0.001, help='clip gradient')

parser.add_argument('-optim', type=str, default='adam')
parser.add_argument('-momentum', type=float, default=0.9)


def main():
    args = parser.parse_args()
    args.cuda = torch.cuda.is_available()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    torch.manual_seed(args.seed)
    if args.seed:
        torch.cuda.manual_seed(args.seed)
    Data = Data_utility(args.id, 0.6, 0.2, device, args)

    wandb.init(project="financial-debug", mode="online")
    args.wb = wandb

    model = eval(args.model)
    model = model.Model(args, Data).to(device)
    # summary(model, (1, 1, 64, 40))
    nParams = sum([p.nelement() for p in model.parameters()])
    print('number of parameters: %d' % nParams)

    optim = makeOptimizer(model.parameters(), args)
    early_stopping_hnn = EarlyStopping(patience=args.patience, delta=args.delta, verbose=args.verbose)
    criterion = nn.CrossEntropyLoss().to(device)

    print('Training start')
    for epoch in range(1, args.epochs + 1):
        epoch_start_time = time.time()
        train_loss = origin_train(Data, Data.train[0], Data.train[1], model, optim, criterion, args)
        val_loss, val_acc = origin_eval(Data, Data.valid[0], Data.valid[1], model, criterion, args)
        print('epoch {:2d} | time used: {:5.2f}s | train_loss {:4.3f} | valid loss {:4.3f} | valid acc {:4.3f}'.format(epoch, (time.time() - epoch_start_time),
                                                                                                    train_loss, val_loss, val_acc))
        args.wb.log({'train_loss': train_loss, 'val_loss': val_loss, 'val_acc': val_acc})
        early_stopping_hnn(val_loss, model)

        if early_stopping_hnn.early_stop:
            print("Early stopping for the phase")
            break
        if epoch % 20 == 0:
            save_model(model, Path(args.model_path, f'checkpoint{epoch}.pt'))
    save_model(model, Path(args.model_path, 'checkpoint.pt'))

    load_model(model, Path(args.model_path, 'checkpoint.pt'))
    test_loss, test_acc = origin_eval(Data, Data.test[0], Data.test[1], model, criterion, args)
    print('accuracy: {} loss:{}'.format(test_acc, test_loss))



In [21]:
args = parser.parse_args()
import os
os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu
main()

usage: ipykernel_launcher.py [-h] [--id ID] [--model MODEL] [--window WINDOW]
                             [--horizon HORIZON] [--normalize NORMALIZE]
                             [--highway_window HIGHWAY_WINDOW]
                             [--kwindow KWINDOW] [-n_rnns N_RNNS]
                             [-rnn_hid_size RNN_HID_SIZE]
                             [--rnn_layers RNN_LAYERS]
                             [-fc_hid_size FC_HID_SIZE] [--dropout DROPOUT]
                             [--output_fun OUTPUT_FUN]
                             [--kernel_type KERNEL_TYPE] [--variance VARIANCE]
                             [--prediction_window_size PREDICTION_WINDOW_SIZE]
                             [--confidence CONFIDENCE] [--hidRNN HIDRNN]
                             [--hidCNN HIDCNN] [--CNN_kernel CNN_KERNEL]
                             [-lr LR] [-epochs EPOCHS]
                             [-epochs_mmd EPOCHS_MMD] [-batch_size BATCH_SIZE]
                             [-lh_scal

Traceback (most recent call last):
  File "/home/qingyi/miniconda3/envs/tf/lib/python3.6/argparse.py", line 2294, in _get_value
    result = type_func(arg_string)
ValueError: invalid literal for int() with base 10: '/home/qingyi/.local/share/jupyter/runtime/kernel-2638c8b7-dbeb-4bed-91d4-c32b74c9d8aa.json'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/qingyi/miniconda3/envs/tf/lib/python3.6/argparse.py", line 1766, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "/home/qingyi/miniconda3/envs/tf/lib/python3.6/argparse.py", line 1972, in _parse_known_args
    start_index = consume_optional(start_index)
  File "/home/qingyi/miniconda3/envs/tf/lib/python3.6/argparse.py", line 1912, in consume_optional
    take_action(action, args, option_string)
  File "/home/qingyi/miniconda3/envs/tf/lib/python3.6/argparse.py", line 1824, in take_action
    argument_values = self._get_values(a

TypeError: must be str, not list

In [25]:
if __name__ == '__main__':
    args = parser.parse_args()
    import os
    os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu
    main()


usage: ipykernel_launcher.py [-h] [--id ID] [--model MODEL] [--window WINDOW]
                             [--horizon HORIZON] [--normalize NORMALIZE]
                             [--highway_window HIGHWAY_WINDOW]
                             [--kwindow KWINDOW] [-n_rnns N_RNNS]
                             [-rnn_hid_size RNN_HID_SIZE]
                             [--rnn_layers RNN_LAYERS]
                             [-fc_hid_size FC_HID_SIZE] [--dropout DROPOUT]
                             [--output_fun OUTPUT_FUN]
                             [--kernel_type KERNEL_TYPE] [--variance VARIANCE]
                             [--prediction_window_size PREDICTION_WINDOW_SIZE]
                             [--confidence CONFIDENCE] [--hidRNN HIDRNN]
                             [--hidCNN HIDCNN] [--CNN_kernel CNN_KERNEL]
                             [-lr LR] [-epochs EPOCHS]
                             [-epochs_mmd EPOCHS_MMD] [-batch_size BATCH_SIZE]
                             [-lh_scal

Traceback (most recent call last):
  File "/home/qingyi/miniconda3/envs/tf/lib/python3.6/argparse.py", line 2294, in _get_value
    result = type_func(arg_string)
ValueError: invalid literal for int() with base 10: '/home/qingyi/.local/share/jupyter/runtime/kernel-2638c8b7-dbeb-4bed-91d4-c32b74c9d8aa.json'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/qingyi/miniconda3/envs/tf/lib/python3.6/argparse.py", line 1766, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "/home/qingyi/miniconda3/envs/tf/lib/python3.6/argparse.py", line 1972, in _parse_known_args
    start_index = consume_optional(start_index)
  File "/home/qingyi/miniconda3/envs/tf/lib/python3.6/argparse.py", line 1912, in consume_optional
    take_action(action, args, option_string)
  File "/home/qingyi/miniconda3/envs/tf/lib/python3.6/argparse.py", line 1824, in take_action
    argument_values = self._get_values(a

TypeError: must be str, not list